In [1]:
%load_ext autoreload
%autoreload 2
import math
from pathlib import Path

import h5py
import numpy as np
from scipy import constants, signal, stats, optimize, integrate, io
from functools import partial
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cycler import cycler

import analysis
from basic.paths import (
    RESULTS_FOLDER,
    PARTICLE_VARIATION_FOLDER,
    DENSITY_VARIATION_FOLDER,
    V_FLOW_VARIATION_FOLDER,
    FOLDER_1D,
    FOLDER_2D,
    MPLSTYLE_FILE
)
from basic import (
    physics,
    Species,
    SpeciesInfo,
    RunInfo,
    Distribution
)

from plots import (
    settings,
    plots_1D,
    plots_2D,
    general,
)
from plots.settings import FIGURE_FULL_SIZE, FIGURE_HALF_SIZE

info = RunInfo(
    electron=SpeciesInfo(
        number_density=12.0e6,
        temperature=100.0,
        charge=-1,
        mass=1.0,
        bulk_velocity=0.0
    ),
    proton=SpeciesInfo(
        number_density=10.0e6,
        temperature=3.0,
        charge=+1,
        mass=1836.152674,
        bulk_velocity=0.0
    ),
    alpha=SpeciesInfo(
        number_density=1.0e6,
        temperature=12.0,
        charge=+2,
        mass=7294.29953,
        bulk_velocity=1.0e5
    )
)

save=False
plt.style.use(MPLSTYLE_FILE)
matplotlib.rcParams['figure.dpi'] = 100

In [ ]:
print(f"{(info.electron.number_density + info.proton.number_density + info.alpha.number_density) * (64 * info.lambda_D) ** 2:.4e}")


In [ ]:
plots_1D.illustrateSimulationGrid(save=True)
plt.show()

In [ ]:
from scipy import special, integrate
def distr_integral(u_alpha, w, m, T):
    c_s = info.c_s
    k = constants.k
    if u_alpha < c_s:
        theta = 0
    else:
        theta = np.arccos(c_s / u_alpha)
    if theta < 1e-15:
        integral_band = special.erf(w * np.sqrt(m / (8 * k * T)))
        return integral_band
    f_geom = w / np.sin(theta)
    e_geom = w / np.cos(theta)

    integral_band = special.erf(w * np.sqrt(m / (8 * k * T)))
    integral_triangle, _ = integrate.quad(
        func=lambda v_x: np.exp(- m * v_x ** 2 / (2 * k * T)) * \
        special.erf((f_geom * (e_geom - 2 * v_x)) / (np.sqrt(8 * k * T / m) * e_geom)) / \
        np.sqrt(8 * np.pi * k * T / m),
        a=0, b=e_geom / 2
    )
    return 2 * integral_band - 4 * integral_triangle
wave_length = 0.5 * 2 * np.pi * info.lambda_D / 0.7
E_field = 0.8
interaction_width = np.sqrt(
    2 * constants.elementary_charge * E_field * wave_length / info.proton.si_mass
)
x = np.linspace(1e5, 2e5, num=1000)
y = np.array([
    distr_integral(u_alpha, w=interaction_width, m=info.alpha.si_mass, T=info.alpha.si_temperature) for u_alpha in x
])
plt.plot(x * 1e-3, y)
plt.xlabel("Initial flow velocity $u_\\alpha$ (km$\\,/\\,$s)")
plt.show()

In [ ]:
def distributionFunction1D(v: float, u: float, n: float, T: float, m: float) -> float:
    from scipy.constants import pi, k
    return np.sqrt(n * m / (2 * pi * k * T)) * np.exp(- m * (v - u) ** 2 / (2 * k * T))

def beamParameterization(
    s: float, t: float, theta: float, omega: float, k: float
) -> tuple[float,float,float]:
    v_x = omega / k * ((1 + t) * np.cos(theta) + s * np.sin(theta))
    v_y = omega / k * ((1 + t) * np.sin(theta) - s * np.cos(theta))
    sqrt_g = omega / k
    return sqrt_g, v_x, v_y

def energyIntegrant(
    s: float, t: float, m: float, theta: float, omega: float, k: float,
) -> float:
    sqrt_g, v_x, v_y = beamParameterization(s, t, theta, omega, k)
    return sqrt_g * (m / 2) * (v_x ** 2 + v_y ** 2)

def waveIntegrant(s: float, t: float,
    theta: float, omega: float, k: float,
    u_x: float, n: float, T: float, m: float,
):
    sqrt_g, v_x, v_y = beamParameterization(s, t, theta, omega, k)
    return sqrt_g * (
        distributionFunction1D(v_x, u_x, n, T, m) *
        distributionFunction1D(v_y, 0.0, n, T, m)
    )

def waveEnergyIntegrant(
    s: float, t: float,
    theta: float, omega: float, k: float,
    u_x: float, n: float, T: float, m: float,
) -> float:
    sqrt_g, v_x, v_y = beamParameterization(s, t, theta, omega, k)
    return sqrt_g * (
        (m / 2) * (v_x ** 2 + v_y ** 2) *
        distributionFunction1D(v_x, u_x, n, T, m) *
        distributionFunction1D(v_y, 0.0, n, T, m)
    )

def integrateComposite(species: Species, u_alpha, theta, omega_max, k_max, interaction_width):
    s_info = info[species]
    func = partial(
        waveEnergyIntegrant,
        theta=theta, omega=omega_max, k=k_max,
        u_x=u_alpha if species == Species.ALPHA else 0.0,
        n=s_info.number_density,
        T=s_info.si_temperature,
        m=s_info.si_mass,
    )
    res, err = integrate.dblquad(func,
        a=-(k_max / omega_max) * interaction_width / 2, b=(k_max / omega_max) * interaction_width / 2,
        gfun=-np.inf, hfun=np.inf, epsabs=1e-20, epsrel=1e-12
    )
    return res, err

def integrateSeparate(species: Species, u_alpha, theta, omega_max, k_max, interaction_width):
    s_info = info[species]
    func_energy = partial(
        energyIntegrant,
        m=s_info.si_mass,
        theta=theta,
        omega=omega_max,
        k=k_max,
    )
    func_wave = partial(
        waveIntegrant,
        theta=theta, omega=omega_max, k=k_max,
        u_x=u_alpha if species == Species.ALPHA else 0.0,
        n=s_info.number_density,
        T=s_info.si_temperature,
        m=s_info.si_mass,
    )
    def _integrant(s):
        f_e = lambda t: func_energy(s=s, t=t)
        f_w = lambda t: func_wave(s=s, t=t)
        return (
            integrate.quad(f_e,
                a=-(k_max / omega_max) * interaction_width / 2,
                b=(k_max / omega_max) * interaction_width / 2,
                epsabs=1e-20, epsrel=1e-12)[0] *
            integrate.quad(f_w,
                a=-(k_max / omega_max) * interaction_width / 2,
                b=(k_max / omega_max) * interaction_width / 2,
                epsabs=1e-20, epsrel=1e-12)[0]
        ) / interaction_width

    res, err = integrate.quad(_integrant, a=-np.inf, b=np.inf, epsabs=1e-20, epsrel=1e-12)
    return res, err

# assumption based on the videos, picked at random
interaction_width = 2 * info.alpha.v_thermal
with h5py.File("theory_u_alpha_dispersion.h5") as f:
    u_alpha_arr = f["u_alpha_bulk"][:]
    theta_arr = f["theta_max"][:]
    k_max_arr = f["k_max"][:]
    omega_max_arr = f["omega_max"][:]
results = []
for u_alpha, theta, k_max, omega_max in zip(u_alpha_arr, theta_arr, k_max_arr, omega_max_arr):
    res_c, err = integrateComposite(Species.ALPHA, u_alpha, theta, omega_max, k_max, interaction_width)
    res_s, err = integrateSeparate(Species.ALPHA, u_alpha, theta, omega_max, k_max, interaction_width)
    results.append(res_c - res_s)

In [ ]:
print(info.c_s)
print(info.electron.v_thermal / np.sqrt(2) * np.sqrt(info.electron.si_mass / info.proton.si_mass) * (1 + 3 * info.proton.temperature / info.electron.temperature))

In [ ]:
plt.plot(u_alpha_arr, results / u_alpha_arr ** 2)
plt.xlabel("Alpha flow velocity $u_\\alpha$")
plt.ylabel("Integral")

In [ ]:
# visualization of distribution overlap with interaction region
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def interactive_plot(u_alpha):
    width = 40
    c_s = info.c_s * 1e-3
    if u_alpha < c_s:
        theta = 0
    else:
        theta = np.arccos(c_s / u_alpha)

    t = np.linspace(0, c_s)
    s = np.linspace(-5 * c_s, 5 * c_s)
    plt.figure()
    # plt.text(0, 0, f"{u_alpha} {theta}")
    circle = plt.Circle((0,0), radius=c_s, fill=False, ls="--", lw=2)
    plt.plot(np.cos(theta) * t, np.sin(theta) * t, ls="solid")
    plt.plot(np.cos(theta) * t, -np.sin(theta) * t, ls="solid")
    plt.plot(c_s * np.cos(theta) + np.sin(theta) * s, c_s * np.sin(theta) - np.cos(theta) * s)
    plt.plot(c_s * np.cos(theta) + np.sin(theta) * s, -c_s * np.sin(theta) + np.cos(theta) * s)
    rect_pos = plt.Rectangle(
        xy=(
            (c_s - width / 2) * np.cos(theta) + np.sin(theta) * (-2 * c_s),
            (c_s - width / 2) * np.sin(theta) - np.cos(theta) * (-2 * c_s)
        ),
        width=1000, height=width, angle=theta * 180 / np.pi+270, alpha=0.4, edgecolor="black"
    )
    rect_neg = plt.Rectangle(
        xy=(
            +(c_s + width / 2) * np.cos(theta) + np.sin(theta) * (-2 * c_s),
            -(c_s + width / 2) * np.sin(theta) + np.cos(theta) * (-2 * c_s)
        ),
        width=1000, height=width, angle=-theta * 180 / np.pi+90, alpha=0.4, edgecolor="black"
    )
    plt.scatter(u_alpha, 0, s=40,zorder=5)
    plt.gca().add_patch(circle)
    plt.gca().add_patch(rect_pos)
    plt.gca().add_patch(rect_neg)
    plt.gca().set_aspect("equal")
    plt.xlim(-110, u_alpha+width)
    plt.ylim(-110, 110)
    plt.xlabel("Velocity $v_{\\alpha,x}$ (km/s)")
    plt.ylabel("Velocity $v_{\\alpha,y}$ (km/s)")
    plt.show()

interactive_plt = interactive(interactive_plot, u_alpha=(100.0, 200.0))
interactive_plt

In [3]:
plots_1D.illustrateVelocitySpace(info, save=True)
plt.show()

In [ ]:
def indicatorRotatedRectangle(x, y, w, h, alpha):
    x_prime = x * np.cos(alpha) + y * np.sin(alpha)
    y_prime = -x * np.sin(alpha) + y * np.cos(alpha)
    return (np.abs(x_prime) <= w / 2) & (np.abs(y_prime) <= h / 2)

def distributionFunction(y, x, w, h, alpha):
    rect1 = indicatorRotatedRectangle(x, y, w, h, alpha)
    rect2 = indicatorRotatedRectangle(x, y, w, h, -alpha)
    return rect1 | rect2

w = 2
h = 1
n_samples = int(2e4)
temp = []
thetas = np.linspace(0, np.pi / 2)
alphas = np. pi / 2 - thetas
for alpha in alphas:
    full = integrate.qmc_quad(
        lambda v: distributionFunction(v[0], v[1], w, h, alpha),
        [-w, -w], [w, w], n_points=n_samples, n_estimates=2)
    norm = full.integral
    t_xx = integrate.qmc_quad(
        lambda v: v[0] ** 2 * distributionFunction(v[0], v[1], w, h, alpha),
        [-w, -w], [w, w], n_points=n_samples, n_estimates=2)
    t_yy = integrate.qmc_quad(
        lambda v: v[1] ** 2 * distributionFunction(v[0], v[1], w, h, alpha),
        [-w, -w], [w, w], n_points=n_samples, n_estimates=2)
    temp.append((t_xx, t_yy))
temp = np.array(temp)[:,:,0]
u_alpha = 65 / np.cos(thetas)
T_eff = np.cos(thetas) * np.mean(temp, axis=1)
plt.plot(
    thetas * 180 / np.pi,
    T_eff / np.max(T_eff),
    label="T")
# plt.plot(
#     angles / 2 * 180 / np.pi,
#     np.cos(angles / 2)[:,None] * temp,
#     label=["$T_{xx}$", "$T_{yy}$"])
plt.axvline(57)
plt.legend()
# plt.xlim(0, 1 / np.cos(60 * np.pi / 180))
plt.show()

In [ ]:
info.electron.si_charge * 45e-9 / info.electron.si_mass / info.omega_pp

In [ ]:
magnetic_field_strength = 4.346348316425957e-06# 45e-9
omega_c_alpha = magnetic_field_strength * info.alpha.si_charge / info.alpha.si_mass
omega_c_proton = magnetic_field_strength * info.proton.si_charge / info.proton.si_mass
print(f"B = {magnetic_field_strength * 1e4}")
print(info.omega_pp / omega_c_alpha)
print(info.omega_pp / omega_c_proton)
print(info.omega_pp / (10 * info.proton.si_charge / info.proton.si_mass))

# Simulation 2D

In [ ]:
# plot max-e-field energy
# TODO: Change markers & colors, perhaps change the format completely so that there is more space for the different markers (in that case focus on 100 and 140 km/s)
plots_2D.magneticFieldDirectionElectricField(info, save=True)
plt.show()

## Alpha flow-speed variation (B=0)

In [ ]:
# plots_2D.maxEnergyVsAlphaFlowSpeed(info, normalize_energy=False)
# plt.show()
plots_2D.maxEnergyVsAlphaFlowSpeed(info, normalize_energy=True, save=save)
plt.show()

In [ ]:
plots_2D.waveAngleVsAlphaFlowSpeed(info, "x", save=True)
plt.show()

In [ ]:
with h5py.File("theory_u_alpha_dispersion.h5") as f:
    theory_v = f["u_alpha_bulk"][:] / 1e3
    theory_omega = f["omega_max"][:] / info.omega_pp
    # print(theory_omega)
    print(f.keys())
    print(f["gamma_max"][:])

In [ ]:
plots_2D.wavenumberVsAlphaFlowSpeed(info, "x", save=True)
plots_2D.omegaVsAlphaFlowSpeed(info, "x", save=True)
plt.show()

In [ ]:
plots_2D.psdOmegaForAlphaFlowSpeed(info, "x", save=True)
plt.show()

In [ ]:
with h5py.File("theory_u_alpha_dispersion_v2.h5") as f:
    u_alpha = f["u_alpha_bulk"][:]
    gamma = f["gamma_max"][:] / info.omega_pp
    theta = f["theta_max"][:] * 180 / np.pi
    k = f["k_max"][:] * info.lambda_D
    omega = f["omega_max"][:] / info.omega_pp

In [ ]:
# plots_1D.linearTheoryDensityRatio(info, save=False)
# plots_1D.linearTheoryFlowVelocity(info, save=False)
plots_1D.linearTheoryWaveProperties(info, save=False)
plt.show()

In [ ]:
# plot 3D temperature vs time for different magnetic fields.
species = Species.PROTON
plt.style.use(MPLSTYLE_FILE)
plt.figure()

leg_u100 = []
leg_u140 = []
for u_label in [100, 140]:
    for folder_id, label in zip(["", "_Bx", "_By", "_Bz", "_Bx_By"], ["B=0", "B_x>0", "B_y>0", "B_z>0", "B_x=B_y"]):
        for filename in sorted((FOLDER_2D / f"v_alpha_bulk_variation{folder_id}").glob("*.h5")):
            u = int(filename.stem[-3:])
            if u != u_label:
                continue
            with h5py.File(filename) as f:
                time = f["Header/time"][:] * info.omega_pp
                temp = np.mean(f[f"Derived/Temperature/{species.value}"], axis=(1,2))
            line = plt.plot(
                time, physics.kelvinToElectronVolt(temp),
                # label=f"(${label}$) $u_\\alpha^{{t=0}}$ = {v}"
                label=f"${label}$"
            )[0]
            (leg_u100 if u_label == 100 else leg_u140).append(line)
leg1 = plt.legend(handles=leg_u100, loc="upper left", title="$u_\\alpha^{t=0}$=100 km/s",ncols=1 if species == Species.ELECTRON else 2, columnspacing=0.5, labelspacing=0.5, fancybox=False, framealpha=0.4)
plt.legend(handles=leg_u140, title="$u_\\alpha^{t=0}$=140 km/s", loc="lower right", ncols=3 if species == Species.ELECTRON else 2, columnspacing=0.5, labelspacing=0.5, fancybox=False, framealpha=0.4)
plt.gca().add_artist(leg1)
plt.xlabel("Time $t\\,\\omega_{pp}$ (1)")
plt.ylabel(f"Temperature $T_{species.symbol()}$ (eV)")
plt.xlim(0, 150.0)
y_low = 1.5 if species == Species.PROTON else 5
plt.ylim(bottom=2 if species == Species.PROTON else 10 if species == Species.ALPHA else 99.7)
plt.tight_layout(pad=0.2)
# plt.savefig(f"figures/svg/simulation-2D/magnetic-field-directions/temperature_{species}-vs-time.svg", bbox_inches="tight")
plt.show()

In [ ]:
for species in Species:
    plots_2D.temperature3DOverTimeForAlphaFlowSpeed(
        info, species
    )

In [ ]:
for species in Species:
    plots_2D.temperatureDifferences3DVsAlphaFlowSpeed(info, species, save=False)
plt.show()

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plots_2D.electricField2DSnapshot(filename, info, time=50.0, save=True)
    plt.show()
    break

In [ ]:
for filename in sorted((FOLDER_2D / "v_alpha_bulk_variation_By").glob("*.h5")):
    plots_2D.electricField2DSnapshot(filename, info, time=50.0)
    plt.show()

In [ ]:
for filename in sorted((FOLDER_2D / "v_alpha_bulk_variation_Bx_By").glob("*.h5")):
    plots_2D.energyEFieldOverTime(filename, info)
    plt.show()

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    # plots_2D.strengthBFieldOverTime(filename, info)
    plots_2D.psdBField(filename, info)
    plots_2D.energyBField(filename, info)
    plt.show()
    break

In [ ]:
for species in [Species.ELECTRON]:
    plots_2D.psdFlowVelocity(info, species, "x", "x")
    plt.show()

In [ ]:
for species in Species:
    plots_2D.flowVelocityVsTime(info, species, "x", "x")
    plt.show()

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plots_2D.energiesOverTime(filename, info)
    plt.show()

In [ ]:
# Kinetic energies in the system
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plt.figure()
    K_species = []
    for species in Species:
        u_alpha = int(filename.stem[-3:])
        with h5py.File(filename) as f:
            time = f["Header/time"][:] * info.omega_pp
            x_grid = f[f"Grid/grid/X"][:]
            y_grid = f[f"Grid/grid/Y"][:]
            px_grid = f[f"Grid/px_py/{species.value}/Px"][:]
            py_grid = f[f"Grid/px_py/{species.value}/Py"][:]
            px_py = f[f'dist_fn/px_py/{species.value}'][:]
        # TODO: Wrong
        u_x, u_y = analysis.flowVelocity2D(x_grid, y_grid, px_grid, py_grid, px_py, info[species])
        K = 0.5 * info[species].si_mass * (u_x ** 2 + u_y ** 2) / constants.electron_volt
        K_species.append(info[species].number_density * K)
    for K_s, species in zip(K_species, Species):
        plt.plot(time, K_s * 1e-6, label=f"$W_{{{species.symbol()}}}$")
    # plt.plot(time[1:], W_total[1:] * 1e-6, label="$W_\\text{total}$")
    plt.yscale("log")
    plt.xlabel("Time $t\\,\\omega_{pp}$ (1)")
    plt.ylabel("Kinetic energy density K (MeV$\\,/\\,$m$^3$)")
    plt.xlim(0.0, 150.0)
    plt.legend(ncols=3, labelspacing=.2, columnspacing=0.5)
    plt.show()
    break

In [ ]:
species = Species.PROTON
files = sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))
labels = [int(p.stem[-3:]) for p in files]
general.momentumDistributionComparison(
    info, species, Distribution.X_PX, legend_ncols=2,
    files=files, labels=labels, times=150.0, legend_title=f"Flow velocity $u_\\alpha$ (km/s)")
plt.show()

In [ ]:
times = [0.0, 40.0, 50, 60.0, 150.0]
species = Species.PROTON
for filename in sorted((FOLDER_2D / "v_alpha_bulk_variation_Bx").glob("*.h5")):
    general.momentumDistributionComparison(info, species, Distribution.Y_PX, filename, times)
    break
plt.show()

In [ ]:
species = Species.ALPHA
dist = Distribution.X_PX
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    general.spaceMomentumDistributon(info, species, dist, filename, time=50.0)
    plt.show()
    break

In [ ]:
species = Species.ALPHA
time = 60.0
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    print(filename.stem[-3:])
    plots_2D.pxPyDistribution(info, species, filename, time)
    plt.show()

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    label = f"u_alpha={filename.stem[-3:]}"
    plots_2D.videoEFieldOverTime(info, filename, "x", label=label, save=True)

In [ ]:
filenames = sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))
species = Species.PROTON
normalized_velocity = True
time_steps = range(0, 1500, 20)
labels = [int(p.stem[-3:]) for p in filenames]
legend_title=f"Flow velocity $u_\\alpha$ (km/s)"
general.videoMomentumDistribution(
    info, Distribution.X_PX, species, filenames,
    save=False, labels=labels, legend_title=legend_title,
    legend_ncols=2)

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    v = f"u_alpha={filename.stem[-3:]}"
    plots_2D.videoPxPyDistribution(info, species=Species.ALPHA, filename=filename, save=True, label=v)

In [ ]:
filename = RESULTS_FOLDER / "epoch_2D/v_alpha_bulk_variation/v_alpha_bulk_106.h5"
v_x, v_y, f_v = general._loadPxPyDistribution(info, Species.PROTON, filename, 100, True)
f_v[f_v<=0] = np.min(f_v[f_v>0])
from scipy import ndimage
plt.contourf(np.log(ndimage.gaussian_filter(f_v, sigma=10)))
plt.gca().set_aspect("equal")

# Simulation 1D

## Density variation

In [ ]:
for s in Species:
    for filename in sorted(DENSITY_VARIATION_FOLDER.glob("density_*.h5")):
        # print(filename)
        ratio = 10 ** float(filename.stem.split("_")[-1])
        n_electron = 12e6
        n_proton = n_electron / (1 + 2 * ratio)
        n_alpha = ratio * n_proton
        info = RunInfo(
            electron=SpeciesInfo(
                number_density=12.0e6,
                temperature=100.0,
                charge=-1,
                mass=1.0,
                bulk_velocity=0.0
            ),
            proton=SpeciesInfo(
                number_density=n_proton,
                temperature=3.0,
                charge=+1,
                mass=1836.152674,
                bulk_velocity=0.0
            ),
            alpha=SpeciesInfo(
                number_density=n_alpha,
                temperature=12.0,
                charge=+2,
                mass=7294.29953,
                bulk_velocity=1.0e5
            )
        )
        # print(np.log10(ratio))
        # plots_1D.energyEFieldOverTime(
        #     filename=filename, info=info
        # )

        with h5py.File(filename) as f:
            time = f["Header/time"][:] * info.omega_pp
            energy = f["Electric Field/ex"][:] ** 2
            temp = physics.kelvinToElectronVolt(f[f"Derived/Temperature/{s.value}"][:])
            plt.errorbar(ratio, np.mean(temp[-10:]) - np.mean(temp[:10]), yerr=np.std(temp[:10]) / np.sqrt(10), marker="o", ls="", color="black")
    plt.title(s)
    plt.xscale("log")
    plt.xlabel("$n_\\alpha/n_\\text{p}$")
    plt.ylabel("Heating $\\Delta T$ (eV)")
    plt.show()
#     energy = np.mean(
#         energy,
#         axis=tuple(range(1, energy.ndim))
#     ) * (constants.epsilon_0 / 2) / (constants.electron_volt)
#     res = analysis.fitGrowthRate(time, energy)
#     plt.plot(ratio, res[0].slope / 2, ls="", marker="o", color="black")
# # import theory
# plt.plot(np.logspace(-2, 1), theory.growthRate(np.logspace(-2, 1)), ls="--")
# plt.xscale("log")

plt.show()

## Run n=8192

In [ ]:
times = [0.0, 40.0, 150.0]
for species, y_bot, x_lim, loc in zip(
    Species,
    [1e-4, 1e-2, 1e-2],
    [(-4, 4), (-4, 10), (-1, 8)],
    [(0.22, 0.05), (0.42, 0.67), (0.3, 0.04)]
):
    # if species == Species.ELECTRON: continue
    general.momentumDistributionComparison(
        info, species, Distribution.X_PX,
        PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5",
        times, legend_loc=loc, legend_title="",
        labels=[f"$t\\,\\omega_\\text{{pp}}={int(t)}$" for t in times],
        y_lim=(y_bot, 1e3 if species == Species.PROTON else None),
        x_lim=x_lim,
        x_ticks=[-4,0,4,8] if species == Species.PROTON else None,
        save=False,
        save_folder="simulation-1D",
    )
plt.show()

In [ ]:
for species, v_lim, v_ticks in zip(
    [Species.PROTON, Species.ALPHA],
    [(-4, 10), (-1, 8)],
    [np.linspace(-4, 8, 4), np.linspace(0, 8, 3)],
):
    general.spaceVelocityDistributionMulti(
        info, species, Distribution.X_PX,
        PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5",
        times=[0.0, 60.0, 150.0],
        v_lim=v_lim,
        v_ticks=v_ticks,
        save=True,
        save_folder="simulation-1D"
    )

In [ ]:
plots_1D.electricFieldOverSpaceAndTime(
    filename=PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5",
    info=info, save=True
)
plt.show()

In [ ]:
for species in Species:
    plots_1D.avgTemperatureXOverTime(
        filename=PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5",
        info=info, species=species, save=True
    )
plt.show()

In [ ]:
plots_1D.velocityDistributionOverTimeCombined(
    PARTICLE_VARIATION_FOLDER / "particles_8192/rep_0.h5",
    info,
    times=[0.0, 150.0],
    v_lim_arr=[(-4, 4), (-4, 10), (-1, 8)],
    v_tick_arr=[[-3,0,3], [-3,0,3,6,9], [0,4,8]],
    save=True,
)

In [ ]:
plt.rcParams["figure.dpi"] = 200
plots_1D.energyEFieldOverTime(
    filename=V_FLOW_VARIATION_FOLDER / "v_alpha_bulk_100.h5",
    info=info, save=False
)
plt.show()

## Particle variation

In [ ]:
plots_1D.particleVariationEnergyVsTime(info=info, save=save)
plt.show()

In [ ]:
for species in Species:
    plots_1D.particleVariationTemperature3D(species, save=save)
plt.show()

In [ ]:
for species in Species:
    plots_1D.particleVariationTemperatureXDiff(info, species, save=save)
plt.show()

In [ ]:
for species in Species:
    plots_1D.particleVariationTemperatureXVsTime(info, species, save=save)
plt.show()

In [ ]:
plots_1D.particleVariationGrowthRate(info, save=save)
plt.show()

In [ ]:
plots_1D.particleVariationWavenumber(info=info, save=save)
plt.show()

In [ ]:
plots_1D.particleVariationFrequency(info=info, save=save)
plt.show()